## 依赖文件pending calc

In [1]:
%run ../kgraph/calc.py


## com.date.name

In [2]:
# com.date.name


sql_dwzd_date  = ''' SELECT   distinct a.DWZD_BH AS '机构编号', a.INST_DATE AS '发生日期', b.y as name
         FROM psx_dw_instdata a , (
              select DWZD_BH as x, DWZD_MC  as y from rpt_dwzd where dwzd_bh in (
           SELECT   distinct a.DWZD_BH as bh
                 FROM psx_dw_instdata a
                 where length(a.DWZD_BH) = 6
         )) b
         where length(a.DWZD_BH) = 6 and a.DWZD_BH = b.x
          '''
results_dwzd_date = query(sql_dwzd_date)

## 获取最低资本向量

In [5]:
# bar3d

import pandas as pd
import numpy as np
# return [ [com,date, dw_name, arr] , ..]
def get_topk(label , keys, k=999999, ):
    # count buy companies of each stock 
    cql = "match (n)-->(out:%s) return count(n) as cc, id(out) as stock_id order by cc desc" % label
    li = graph.data(cql)
    stock_ids = []
    stock_nums = []
    for i in li:
        stock_ids.append(i['stock_id'])
        stock_nums.append(i['cc'])


    # 超级慢！！ 20 mins
    com_date_stock300=[]

    min_capital_keys = keys


    for com, date, dw_name in results_dwzd_date[:]:
        # print(com,date, dw_name)
        # arr 表示feature的值，k维向量； eg 最低资本
        arr = []
        # 股票top k
        for i in stock_ids[:k]:
            # 当前 i 股票，单位时间购买的最低资本
#             cql = "match (c:company{dwzd_bh :'%s'})-->(d:date{inst_date: '%s'})-[*]->(oa:own_amount)-->(s:%s)\
#             where id(s) = %d \
#             return oa " %(com, date, label, i)
            cql = "match (oa:own_amount)-->(s:%s) where oa.tmp_dwzd_bh = '%s' \
                and oa.tmp_inst_date = '%s' and id(s) = %d  return oa" %( label,com, date, i)
            cur = graph.run(cql)
            current_stock_sum = 0
            while cur.forward():
                oa = cur.current().values()[0]
                # 找到当前最低资本key
                key = (  set(dict(oa).keys()) & set(min_capital_keys)  ).pop()
                if key is None:
                    continue
                else:
                    current_stock_sum += float(oa[key])
            else:
                arr.append(current_stock_sum)
        com_date_stock300.append([com,date, dw_name, arr])
    
    stock_dic = {}
    for i in com_date_stock300:
        stock_dic ['-'.join(i[:3])] = i[3]
    
    df = pd.DataFrame(  stock_dic , index= stock_ids[:k])
    df.to_csv("./"+label+'.csv')
        
    return com_date_stock300, df


# keys = [
#     "EquityPriceRiskStockNoHedgingAndUnqualifiedHedgingMinimumCapital",
#     "EquityPriceRiskStockQualifiedHedgingMinimumCapitalOfHedgingPortfolio",
#     "EquityPriceRiskUnlistedEquitiesMinimumCapital",
#     "EquityPriceRiskPreferredSharesMinimumCapital",
#     "EquityPriceRiskShortPositionOfStockIndexFuturesOutOfEffectivenessMinimumCapital",
# ]   

# tt , ddf= get_topk('stock' ,keys= keys, k= 9999)      
# print(tt)

In [7]:
import numpy as np
import matplotlib.pyplot as plt
def calc_similarity(com_date_stock300, distance = 'cos' ):
    # 计算相似度
    similarity = []
    for i,vec1 in enumerate(com_date_stock300):
        k = i+1
        # if i >2: break
        for j in range(len(com_date_stock300))[k:]:
            vec2 = com_date_stock300[j]
            vector1 = np.array([i+0.00000000001 for i in vec1[3]])
            vector2 = np.array([i+0.00000000001 for i in vec2[3]])
            cos =np.dot(vector1,vector2)/ (np.linalg.norm(vector1)*(np.linalg.norm(vector2)))  
            similarity.append([vec1[:3], vec2[:3], cos])

    similarity = sorted(similarity, key = lambda x: x[2], reverse = True)


    simi = []
    for s in similarity:
        simi.append(s[2])



    x = np.arange(0, len(simi));
    y = np.array(simi)
    plt.plot(x, y)
    plt.show()
    return similarity


# print(tt[0])
# print([i[:3] + i[3][:300] for i in tt[:3]])
# si_t = calc_similarity([i[:3] + [i[3][:23]] for i in tt])

In [6]:


MinimumCapital_keys = {
    
    # "1.外汇 forex MR27 MR28 MR29 hold" 
    'forex':[
        "ExchangeRateRiskForeignCurrencyDenominatedAssetsLiabilitiesUnhedgedOrIneffectivelyHedgedMinimumCapital",
        "ExchangeRateRiskForeignCurrencyDenominatedAssetsLiabilitiesHighlyEffectivelyHedgedMinimumCapitalOfHedgingPortfolio",
        "ExchangeRateRiskForeignExchangeForwardAssetsLiabilitiesIneffectivelyHedgedMinimumCapitalOfForeignExchangeForwardContracts"],

    # "10.股权 equity MR11 MR15 MR17 MR22 hold"
    'equity':[
        "EquityPriceRiskUnlistedEquitiesMinimumCapital",
        "EquityPriceRiskInfrastructureEquityInvestmentPlansMinimumCapital",
        "EquityPriceRiskUnlistedEquityInvestmentPlansMinimumCapital",
        "EquityPriceRiskLongTermEquityInvestmentsInSubsidiariesJointVenturesAndAssociatedCompaniesMinimumCapital",
    ],
    # 8.股票 stock MR09 MR10 MR19 MR20 MR21 buy
    "stock":[
        "EquityPriceRiskStockNoHedgingAndUnqualifiedHedgingMinimumCapital",
        "EquityPriceRiskStockQualifiedHedgingMinimumCapitalOfHedgingPortfolio",
        "EquityPriceRiskUnlistedEquitiesMinimumCapital",
        "EquityPriceRiskPreferredSharesMinimumCapital",
        "EquityPriceRiskShortPositionOfStockIndexFuturesOutOfEffectivenessMinimumCapital",
    ]   ,

    # 2.产品 product CR06 CR08 CR09 CR10 CR11 CR12 CR13   MR08 MR16  MR18 buy
    "product":[
        "CounterpartyDefaultRiskAssetSecuritizationProductsMinimumCapital",
        "CounterpartyDefaultRiskTrustPlansMinimumCapital",
        "CounterpartyDefaultRiskTrustPlansMinimumCapitalOfUnderlyingAssets",
        "CounterpartyDefaultRiskAssetManagementProductsMinimumCapital",
        "CounterpartyDefaultRiskInfrastructureDebtInvestmentPlansMinimumCapital",
        "CounterpartyDefaultRiskPropertyDebtInvestmentPlansMinimumCapital",
        "CounterpartyDefaultRiskProjectAssetBackedPlansMinimumCapital",
        "InterestRateRiskOtherFixedIncomeProductsMinimumCapital",
        "EquityPriceRiskAssetManagementProductsMinimumCapital",
        "EquityPriceRiskEquityTrustPlansMinimumCapital",
    ],

    # 3.合约 contract MR05 CR14  hold
    "contract":[
        "CounterpartyDefaultRiskForwardForeignExchangeContractsAndInterestRateSwapsForPurposeOfHedgingMinimumCapital",
        "InterestRateRiskInterestRateDerivativesInterestRateSwapsMinimumCapital",
    ],

    # 4.再保险 reinsurance CR15 CR16 CR17 hold
    "reinsurance":[
        "CounterpartyDefaultRiskOutwardReinsuranceAssetsMinimumCapital",
        "CounterpartyDefaultRiskOutwardReinsuranceAssetsMinimumCapital",
        "CounterpartyDefaultRiskInwardReinsuranceAssetsMinimumCapital",
    ],

    # 5.债券 bond MR03 CR05 buy
    "bond":[
        "CounterpartyDefaultRiskEnterpriseBondsCorporateBondsMinimumCapital",
        "InterestRateRiskBondAssetsNoHedgingAndUnqualifiedHedgingMinimumCapital",
    ],

    # 6.资产 assets CR01 buy
    "assets":[
        "CreditRiskMinimumCapitalForSpreadRisk",
    ],

    # 7.房地产 real estate MR23 MR24 hold
    "real_estate":[
        "MarketRiskMinimumCapitalForPropertyPriceRisk",
        "MarketRiskMinimumCapitalForPropertyPriceRisk",
    ],

    # 9.基金 fund MR12 MR13 buy
    "fund":[
        "EquityPriceRiskSecuritiesInvestmentFundMinimumCapital",
        "EquityPriceRiskSecuritiesInvestmentFundMinimumCapital",
    ],



    # 11.境外 overseas MR25 MR26 buy
    "overseas":[
        "MarketRiskMinimumCapitalForOverseasFixedIncomeInvestmentAssetsPriceRisk",
        "MarketRiskMinimumCapitalForOverseasEquityInvestmentAssetsPriceRisk",
    ],

    # 12.可转债 convertible bond  MR14 buy
    "convertible_bond":[
        "EquityPriceRiskConvertibleBondMinimumCapital",
    ],
    # 13.证券 security MR04 MR06 MR07 hold
    "security":[
        "InterestRateRiskAssetSecuritizationProductsMinimumCapital",
        # 下面有两个最低资本
        "InterestRateRiskInterestRateDerivativesGovernmentBondFuturesQualifiedHedgingMinimumCapitalOfHedgedBondPortfolio",    
        "InterestRateRiskInterestRateDerivativesGovernmentBondFuturesQualifiedHedgingMinimumCapitalOfHedgingPortfolio",
        "InterestRateRiskInterestRateDerivativesGovernmentBondFuturesUnqualifiedHedgingMinimumCapital",
    ],


}

MinimumCapital_arrays = {}
MinimumCapital_dfs = {}

for label, keys in MinimumCapital_keys.items():
    
    MinimumCapital_arrays[label], MinimumCapital_dfs[label] = get_topk(label, keys, k = 9999)

## 获取风险暴露和最低资本

In [8]:


def exposure_cost(label , k=999999, ):
    # count buy companies of each stock 
    cql = "match (o:own_amount)-->(out:%s) \
    where any ( i in keys(o) where  i =~ '.*PurchaseCost.*' )           \
    and any ( i in keys(o) where  i =~ '.*RiskExposure.*' )  \
    return count(o) as cc, \
    id(out) as stock_id order by cc desc" % label
    li = graph.data(cql)
    stock_ids = []
    stock_nums = []
    for i in li:
        stock_ids.append(i['stock_id'])
        stock_nums.append(i['cc'])

    # 超级慢！！ 20 mins
    expo_out=[]
    cost_out=[]

    for com, date, dw_name in results_dwzd_date[:]:
        # arr 表示feature的值，k维向量； eg 最低资本
        arr1 = []
        arr2 = []
        # 股票top k
        for i in stock_ids[:k]:
            # 当前 i 股票，单位时间购买的最低资本
#             cql = "(oa:own_amount)-->(s:%s)\
#             where id(s) = %d \
#             return oa " %(com, date, label, i)
            
            cql = "match (oa:own_amount)-->(s:%s) where oa.tmp_dwzd_bh = '%s' \
                and oa.tmp_inst_date = '%s' and id(s) = %d  return oa" %( label,com, date, i)
            cur = graph.run(cql)
            expo_sum = 0
            cost_sum = 0
            while cur.forward():
                oa = cur.current().values()[0]
                # 找到当前最低资本key
                # key = (  set(dict(oa).keys()) & set(min_capital_keys)  ).pop()
                key1 = [ kk for kk in dict(oa).keys() if "RiskExposure" in kk ]
                key2 = [ kk for kk in dict(oa).keys() if "PurchaseCost" in kk ]
                if key1 == []:
                    continue
                else:
                    key1 = key1[0]
                if key2 == []:
                    continue
                else:
                    key2 = key2[0]
                expo_sum += float(oa[key1])
                cost_sum += float(oa[key2])
                
            else:
                arr1.append(expo_sum)
                arr2.append(cost_sum)
        expo_out.append([com,date, dw_name, arr1])
        cost_out.append([com,date, dw_name, arr2])
    
    dic1 = {}
    dic2 = {}
    for i in expo_out:
        dic1 ['-'.join(i[:3])] = i[3]
    for i in cost_out:
        dic2 ['-'.join(i[:3])] = i[3]
    
    df = pd.DataFrame(  dic1 , index= stock_ids[:k])
    df.to_csv("./expo_"+label+'.csv')
    df = pd.DataFrame(  dic2 , index= stock_ids[:k])
    df.to_csv("./cost_"+label+'.csv')
        

ex_cost_labels = [
    "equity",
    "stock",
    "convertible_bond",
    
    "overseas",
    "product",
    "fund",
]  

for exlabel in ex_cost_labels:
    exposure_cost(exlabel)